
<img src="http://biopython.org/DIST/docs/tutorial/images/biopython_logo.svg" width=500 height=500 />

## Overview

“Basically, we just like to program in Python and want to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and scripts.” 
(Biopython documentation)


- Parsers for many bioinformatics file formats
    - **PDB**
    - Fasta
    - BLAST
    - ClustalW
    - GenBank
- Easy access to many online services
    - NCBI
    - Expasy
- Interface to other not-so-common used programs
    - ClustalW
    - DSSP
    - MSMS

## Installation

Not covered here today, but you can access the biopython [github](https://github.com/biopython/biopython) or [documentation](https://biopython.org/wiki/Documentation) and follow the instructions there.

Don't worry. It should be as simple as running one of the two commands below:

***pip install biopython***

 

***conda install biopython***

### Loading libraries

- ***PDBParser***: module to parse pdb files
- ***PDBList***: module to help accessing PDB structures (locally or directly from the PDB)
- ***NeighborSearch***: methods for searching the 3D space
- ***Selection***: helps with selecting specific data structures

In [ ]:
from Bio.PDB import PDBParser, PDBList, NeighborSearch, Selection

import warnings
warnings.filterwarnings('ignore') # Ignoring warning messages

### Define PDB code and output directory

In [ ]:
pdb_code = "1CSE"
output_dir = '.'

### Using Biopython to retrieve protein structure from the PDB

In [ ]:
pdbl = PDBList()
pdb_file = pdbl.retrieve_pdb_file(pdb_code=pdb_code, file_format='pdb', pdir=output_dir)

### Parsing structure to a Biopython object

In [ ]:
parser = PDBParser()
structure = parser.get_structure(pdb_code,pdb_file)

### Printing header information

In [ ]:
for key,value in structure.header.items():
    print("{}:\t{}\n".format(key,value))

### Model, Chain, Residues and Atoms

In [ ]:
model = structure[0]
chains = list(model.get_chains())
print(chains)

In [ ]:
chain_E = chains[0]
chain_I = chains[1]

In [ ]:
residues_chain_E = list(chain_E.get_residues())

In [ ]:
residues_chain_I = list(chain_I.get_residues())

In [ ]:
residue1 = residues_chain_E[1]
print(residue1.resname)

In [ ]:
atoms_residue = list(residue1.get_atoms())
print(atoms_residue)
atoms_residue[0].get_coord()

Methods for the **Residue** object

In [ ]:
residue1.get_resname() 

In [ ]:
residue1.get_id() # (Hetero-field, sequence_identifier, insertion_code)

In [ ]:
list(residue1.get_atoms())

Accessing specific atoms

In [ ]:
print(residue1['C'].get_coord())
print(residue1['CA'].get_coord())

In [ ]:
residue2 = residues_chain_E[10]
print(residue2['C'].get_coord())
print(residue2['CA'].get_coord())

Quickly calculating distance between two atoms

In [ ]:
print(residue1['CA']-residue2['CA'])

Methods for  **Atom** object

In [ ]:
a = residue1['CA']

In [ ]:
a.get_name() # atom name (spaces stripped, e.g. "CA")

In [ ]:
a.get_id() # id (equals atom name)

In [ ]:
a.get_coord() # atomic coordinates

In [ ]:
a.get_bfactor() # isotropic B factor

In [ ]:
a.get_occupancy() # occupancy

In [ ]:
a.get_altloc() # alternative location specifier

In [ ]:
a.get_fullname() # atom name (with spaces, e.g. ".CA.")

### Selecting interface residues
#### Here we will use the definition of an interface residues as all residues at maximum 5A away from the other protein chain

In [ ]:
parser = PDBParser()
structure = parser.get_structure('myPDB',pdb_file)
print(structure)
model = structure[0]
chains = list(model.get_chains())
print(chains)

In [ ]:
chain1 = chains[0]
chain2 = chains[1]

In [ ]:
residues_interface_chain1 = set() # variable to store list of interface residues for chain1
residues_interface_chain2 = set() # variable to store list of interface residues for chain2
THRESHOLD_DISTANCE = 5.0 # Defining threshold distance

In [ ]:
atoms_list = Selection.unfold_entities(chain2,'A') # Selecting all atoms from chain2
ns = NeighborSearch(atoms_list) # Defining search space with the selected atoms

for atom in chain1.get_atoms(): # For each atom in chain1
    center = atom.get_coord() # define it as a center point
    neighbors = ns.search(center,THRESHOLD_DISTANCE,level='R') # search for any residue within 5A or less from the center
    
    if len(neighbors) != 0: # if the search return anything
        residues_interface_chain1.update(neighbors) # add it to the list of interface residues for chain1

In [ ]:
residues_interface_chain1

In [ ]:
atoms_list = Selection.unfold_entities(chain1,'A') # Selecting all atoms from chain1
ns = NeighborSearch(atoms_list) # Defining search space with the selected atoms

for atom in chain2.get_atoms(): # For each atom in chain2
    center = atom.get_coord() # define it as a center point
    neighbors = ns.search(center,THRESHOLD_DISTANCE,level='R') # search for any residue within 5A or less from the center
    
    if len(neighbors) != 0: # if the search return anything
        residues_interface_chain2.update(neighbors) # add it to the list of interface residues for chain1

In [ ]:
residues_interface_chain2

### Create selection for visualisation with NGLviewer

Not covered here today, but you can access the nglview [github](https://github.com/nglviewer/nglview) and follow the instructions there.

Again, it should be as simple as running one of the two commands below:

***pip install nglview***

 

***conda install nglview -c conda-forge***

In [ ]:
import nglview # Importing library

In [ ]:
print(pdb_file)

In [ ]:
viewer = nglview.show_file(pdb_file)
viewer

In [ ]:
viewer.color_by('sstruc') # colour by type of secondary structure (alpha helices, beta sheets and loops)

In [ ]:
viewer.color_by('element') # colour by atom element

In [ ]:
viewer.remove_cartoon()
viewer.add_ball_and_stick()

In [ ]:
viewer.clear()

In [ ]:
viewer.add_ball_and_stick(selection='protein')

In [ ]:
viewer.clear()

In [ ]:
viewer.add_cartoon(selection='protein')

Colouring options
- bfactor
- chainid
- element
- sstruc
- hydrophobicity
- electrostatic
...

[Complete guide](http://nglviewer.org/ngl/api/manual/coloring.html)

### Selecting interface residues

#### Here we are adding all selected residues except for water molecules

In [ ]:
interface_selection_chain1 = []
interface_selection_chain2 = []

for res in residues_interface_chain1: # for each interface residue on chain 1
    if res.id[0] != 'W': # select only if it is not a water molecule
        interface_selection_chain1.append("{}:{}".format(res.id[1],res.parent.id))
        
for res in residues_interface_chain2: # for each interface residue on chain 2
    if res.id[0] != 'W': # select only if it is not a water molecule
        interface_selection_chain2.append("{}:{}".format(res.id[1],res.parent.id))

In [ ]:
print(interface_selection_chain1)
print(interface_selection_chain2)

In [ ]:
interface_selection_chain1 = " or ".join(interface_selection_chain1)
interface_selection_chain2 = " or ".join(interface_selection_chain2)
print(chain1)
print(interface_selection_chain1)
print(chain2)
print(interface_selection_chain2)

In [ ]:
viewer.add_representation('ball+stick', selection=interface_selection_chain1, color="magenta")
viewer.add_representation('ball+stick', selection=interface_selection_chain2, color="#76ff03")

List of representations available for nglviewer
- ball+stick
- cartoon
- line
- spacefill
- surface

Remove representations

In [ ]:
viewer.remove_spacefill() # specific to spacefill
viewer.remove_ball_and_stick() # specific to ball+stick

viewer.clear_representations() # generic

Return to previous representations

In [ ]:
viewer.add_cartoon(selection='protein')

In [ ]:
viewer.add_representation('ball+stick', selection=interface_selection_chain1, color="magenta")
viewer.add_representation('ball+stick', selection=interface_selection_chain2, color="#76ff03")

In [ ]:
viewer.add_surface(selection="protein", opacity=0.3)

In [ ]:
viewer.remove_surface()

In [ ]:
viewer.add_surface(selection="protein", opacity=1)

In [ ]:
viewer.remove_surface()

For more information and examples visit the [github](https://github.com/nglviewer/nglview) or the [documentation](http://nglviewer.org/nglview/latest/api.html).